### Atividade 1 - Cite 5 diferenças entre o AdaBoost e o GBM

AdaBoost vs GBM

1. AdaBoost utiliza uma floresta de *stumps*, que são árvores de decisão com apenas 1 de profundidade e 2 folhas. GBM utiliza uma floresta de árvores de decisão com a possibilidade de serem completas ou podadas.  
2. AdaBoost tem como primeiro passo a seleção de um *stump* com o melhor desempenho. No GBM o primeiro passo é o cálculo da média do Y (variável resposta). 
3. No AdaBosst cada resposta tem um peso diferente para a agregação final da predição.	No GBM a predição das respostas das árvores é calculada através de um multiplicador em comum chamado `learning_rate` (eta).
4. No AdaBoost as árvores (*stumps*) são influenciadas entre si sequencialmente e os datasets sequenciais são criados por Booststrap com peso, baseado no cálculo de performance do melhor *stump*, onde exemplos classificados incorretamente têm maior probabilidade de serem selecionados. No GBM as novas árvores são criadas a partir do resíduo da predição da árvore anterior (o primeiro resíduo é calculado a partir da média de Y).
5. AdaBoost utiliza como predição final uma votação majoritária ponderada das respostas de acordo com a performance de cada *stump*. No GBM a predição é baseada no ajuste do modelo através da redução dos erros residuais.

### Atividade 2 - Acesse o [link Scikit-learn–adaboost](https://scikit-learn.org/stable/modules/ensemble.html), leia a explicação (traduza se for preciso) e crie um jupyternotebook contendo o exemplo de classificação e de regressão do GBM

In [3]:
#Classificação

from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

clf = GradientBoostingClassifier(n_estimators=100, 
                                 learning_rate=1.0, 
                                 max_depth=1, 
                                 random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.913

In [2]:
#Regressão

import numpy as np
from sklearn.metrics  import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor

X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
X_train, X_test = X[:200], X[200:]
y_train, y_test = y[:200], y[200:]

est = GradientBoostingRegressor(n_estimators=100, 
                                learning_rate=0.1, 
                                max_depth=1, 
                                random_state=0, 
                                loss='squared_error').fit(X_train, y_train)

mean_squared_error(y_test, est.predict(X_test))

5.009154859960321

### Atividade 3 - Cite 5 Hyperparametros importantes no GBM

1. `n_estimators`: controla o número de árvores no modelo (valor maior pode aumentar tanto o tempo de treinamento quanto a capacidade de aprendizado do modelo)
2. `learning_rate`: deve estar no intervalo de 0.0 a 1.0; controla a taxa de aprendizado do modelo e interage fortemente com o número de estimadores (`n_estimators`) 
3. `max_depth`: controla a profundidade máxima de cada árvore no modelo, especificando o número máximo de nós
4. `max_features`: controla o número de variáveis consideradas ao fazer uma divisão em cada nó das árvores
5. `criterion`: mede a qualidade de quebra dos nós das árvores

### Atividade 4 - Utilize o GridSearch para encontrar os melhores hyperparametros para o conjunto de dados do exemplo 

In [6]:
from sklearn.model_selection import GridSearchCV
import pandas as pd

In [9]:
%%time

X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

clf = GradientBoostingClassifier()
params = {
    'n_estimators':list(range(1, 1000, 100)),
    'learning_rate': [i/10 for i in range(1, 11)],
    'max_depth': [1] 
}

grid_clf = GridSearchCV(estimator=clf,
                        param_grid=params,
                        scoring='accuracy',
                        cv=5)

grid_clf.fit(X_train, y_train)

print(grid_clf.best_estimator_)
print(grid_clf.score(X_test, y_test))

GradientBoostingClassifier(learning_rate=0.8, max_depth=1, n_estimators=501)
0.9446
CPU times: user 5min 45s, sys: 170 ms, total: 5min 45s
Wall time: 5min 47s


### Atividade 5 -  Acessando o artigo do Jerome Friedman [(Stochastic)](https://jerryfriedman.su.domains/ftp/stobst.pdf) e pensando no nome dado ao Stochastic GBM, qual é a maior diferença entre os dois algoritmos?

Considerando o nome dado ao Stochastic GBM e com referência à teoria probabilística, padrão estocástico tem origem em eventos aleatórios; podemos então descrever o modelo como um algoritmo que incorpora variáveis aleatórias.<br> 
O Stochastic GBM é uma combinação do Gradient Boosting e Bootstrap Aggregating, sendo considerado um híbrido das técnicas Bagging e Boosting. Em cada iteração, o classificador base é treinado em um subconjunto aleatório e não repetitivo dos dados de treinamento, utilizando em média metade da amostra. Essa aleatoriedade na amostragem do conjunto de dados em cada iteração de treino (como o Bootstrap, porém sem reposição) melhora significativamente a precisão do Gradient Boosting e torna o modelo mais robusto em comparação com o GBM tradicional. Isso ocorre porque a aleatoriedade ajuda a evitar o overfitting e promove uma melhor generalização do modelo.